In [1]:
import numpy as np
import time
import csv
import gillespy2
from gillespy2 import Model, Species, Parameter, Reaction, Event, \
                      EventTrigger, EventAssignment, RateRule, \
                      AssignmentRule, FunctionDefinition, TimeSpan
# from gillespy2 import ODECSolver
from gillespy2 import SSACSolver
# from gillespy2 import CLESolver
# from gillespy2 import TauLeapingCSolver
# from gillespy2 import TauHybridCSolver
from distributed import Client
from distributed.diagnostics.progressbar import progress

In [2]:
client = Client(n_workers=8) 

/home/ubuntu/.local/lib/python3.8/site-packages/distributed/dashboard/core.py:20: UserWarning: 
Dask needs bokeh >= 2.4.2, < 3 for the dashboard.
You have bokeh==3.0.3.
Continuing without the dashboard.
  warnings.warn(
2023-03-19 14:34:46,696 - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2023-03-19 14:34:46,702 - distributed.scheduler - INFO - State start
2023-03-19 14:34:46,730 - distributed.scheduler - INFO -   Scheduler at:     tcp://127.0.0.1:35299
2023-03-19 14:34:46,731 - distributed.scheduler - INFO -   dashboard at:            127.0.0.1:8787
2023-03-19 14:34:46,789 - distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:36981'
2023-03-19 14:34:46,801 - distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:38641'
2023-03-19 14:34:46,807 - distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:35149'
2023-03-19 14:34:46,821 - dis

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 15.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35299,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.63 GiB
Comm: tcp://127.0.0.1:41517,Total threads: 1
Dashboard: http://127.0.0.1:40253/status,Memory: 1.95 GiB
Nanny: tcp://127.0.0.1:36981,


In [4]:
class PredatorPrey1(Model):
    def __init__(self, tspan):
        Model.__init__(self, name="Predator-Prey(1)")
        self.volume = 1

        # Parameters
        r_birth = Parameter(name="r_birth", expression="10")
        f_birth = Parameter(name="f_birth", expression="0.01")
        f_death = Parameter(name="f_death", expression="10")
        self.add_parameter([r_birth, f_birth, f_death])

        # Variables
        rabbit = Species(name="rabbit", initial_value=1000, mode="discrete")
        fox = Species(name="fox", initial_value=1000, mode="discrete")
        self.add_species([rabbit, fox])

        # Reactions
        rabbit_birth = Reaction(name="rabbit_birth", reactants={'rabbit': 1}, products={'rabbit': 2}, rate="r_birth")
        fox_birth = Reaction(name="fox_birth", reactants={'rabbit': 1, 'fox': 1}, products={'fox': 2}, rate="f_birth")
        fox_death = Reaction(name="fox_death", reactants={'fox': 1}, products={}, rate="f_death")
        self.add_reaction([rabbit_birth, fox_birth, fox_death])

        # Timespan
        self.timespan(tspan)

In [5]:
def configure_simulation(n, tspan_n):
    tspan = TimeSpan(np.arange(0, tspan_n, 0.01))
    model = PredatorPrey1(tspan)
    solver = SSACSolver(model=model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":n,
        "timeout":20,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

In [6]:
# What value(s) do you want to extract from the simulation trajectory
# i = 0: fox; i = 1: rabbit
def population_at_last_timepoint(c, res, i):
    if c.verbose:
        print(f'population_at_last_timepoint {c.variable_of_interest[i]}={res[c.variable_of_interest[i]][-1]}')
        #print(res)
        #print(res[c.variable_of_interest])
    return res[c.variable_of_interest[i]][-1]

In [7]:
# How do we combine the values from multiple trajectores
def average_of_ensemble(c, data):
    a = np.average(data)
    if c.verbose:
        print(f'average_of_ensemble = {a}')
    return a

In [8]:
class ParameterSweep3D():
    
    def write_to_file(c, filename):
        with open(filename, 'a', encoding='UTF8', newline = "") as f:
            writer = csv.writer(f)
            #row = ["runtime", "The number of rabbits", c.p1, c.p2, c.p3, "number_of_trajectories"]
            #writer.writerow(row)
            for i in range(len(c.para)):
                row = []
                row.append(c.runtime[i])
                row.append(c.data_rabbit[i])
                row.append(c.data_fox[i])
                for j in range(len(c.para[0])):
                    row.append(c.para[i][j])
                row.append(c.number_of_trajectories)
                row.append(c.tspan_n)
                writer.writerow(row)
                
    def plot(c):
        from matplotlib import pyplot as plt
        from mpl_toolkits.axes_grid1 import make_axes_locatable
        fig, ax = plt.subplots(figsize=(8, 8))
        plt.imshow(c.data)
        ax.set_xticks(np.arange(c.data.shape[1]) + 0.5, minor=False)
        ax.set_yticks(np.arange(c.data.shape[0]) + 0.5, minor=False)
        plt.title(f'Parameter Sweep - Variable: {c.variable_of_interest}')
        ax.set_xticklabels(c.p1_range, minor=False, rotation=90)
        ax.set_yticklabels(c.p2_range, minor=False)
        ax.set_xlabel(c.p1, fontsize=16, fontweight='bold')
        ax.set_ylabel(c.p2, fontsize=16, fontweight='bold')
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.2)
        _ = plt.colorbar(ax=ax, cax=cax)


    def plotplotly(c, return_plotly_figure=False):
        from plotly.offline import init_notebook_mode, iplot
        import plotly.graph_objs as go

        xaxis_ticks = c.p1_range
        yaxis_ticks = c.p2_range

        trace_list = [go.Heatmap(z=c.data, x=xaxis_ticks, y=yaxis_ticks)]
        title = dict(text=f'<b>Parameter Sweep - Variable: {c.variable_of_interest}</b>', x=0.5)
        xaxis_label = dict(title=f'<b>{c.p1}</b>')
        yaxis_label = dict(title=f'<b>{c.p2}</b>')

        layout = go.Layout(title=title, xaxis=xaxis_label, yaxis=yaxis_label)

        fig = dict(data=trace_list, layout=layout)

        if return_plotly_figure:
            return fig
        iplot(fig)

In [9]:
def run(c, verbose=False):
    c.verbose = verbose
    fn = c.feature_extraction
    ag = c.ensemble_aggragator
    data_rabbit = np.zeros((len(c.p1_range), len(c.p2_range), len(c.p3_range)))
    data_fox = np.zeros((len(c.p1_range), len(c.p2_range), len(c.p3_range)))
    runtime = np.zeros((len(c.p1_range), len(c.p2_range), len(c.p3_range)))
    para = []
    c.number_of_trajectories = c.kwargs["number_of_trajectories"]
    c.timeout = c.kwargs["timeout"]
    for i, v1 in enumerate(c.p1_range):
        for j, v2 in enumerate(c.p2_range):
            for u, v3 in enumerate(c.p3_range):
                if c.verbose:
                    print(f'running {c.p1}={v1}, {c.p2}={v2}, {c.p3}={v3}')
                        #para = np.append(para, [[v1, v2]], axis = 0)
                para.append([v1, v2, v3])
                start = time.time()
                if(c.number_of_trajectories > 1):
                    tmp_results = c.model.run(**c.kwargs, variables={c.p1:v1, c.p2:v2, c.p3:v3})
                    data_fox[i, j, u] = ag(c,[fn(c,x,0) for x in tmp_results])
                    data_rabbit[i, j, u] = ag(c,[fn(c,x,1) for x in tmp_results])
                        
                else:
                    tmp_results = c.model.run(**c.kwargs, variables={c.p1:v1, c.p2:v2, c.p3:v3})
                    data_fox[i, j, u] = c.feature_extraction(c,tmp_results,0)
                    data_rabbit[i, j, u] = c.feature_extraction(c,tmp_results,1)
                end = time.time()
                if c.verbose:
                    print(f"running time: {end-start}")
                runtime[i, j, u] = end - start
                        
    c.data_fox = data_fox.flatten()
    c.data_rabbit = data_rabbit.flatten()
    c.runtime = runtime.flatten()
    c.para = np.array(para)
    return c

In [10]:
class ParameterSweepConfig3D(ParameterSweep3D):
    def __init__(self, min_num_1, max_num_1, min_num_2, max_num_2, min_num_3, max_num_3, p1_steps, p2_steps, p3_steps, tspan_n, kwargs):
        self.min_num_1 = min_num_1
        self.max_num_1 = max_num_1
        self.min_num_2 = min_num_2
        self.max_num_2 = max_num_2
        self.min_num_3 = min_num_3
        self.max_num_3 = max_num_3
        self.kwargs = kwargs
    
        # What class defines the GillesPy2 model
        tspan = TimeSpan(np.arange(0, tspan_n, 0.01))
        self.tspan_n = tspan_n
        self.model = PredatorPrey1(tspan)
        #self.model = PredatorPrey1()
        # ENTER PARAMETER 1 HERE
        self.p1 = 'r_birth'
        # ENTER PARAMETER 2 HERE
        self.p2 = 'f_birth'
        # ENTER PARAMETER 3 HERE
        self.p3 = 'f_death'
        # ENTER START VALUE FOR P1 RANGE HERE
        self.p1_min = self.min_num_1 * float(eval(self.model.get_parameter(self.p1).expression))
        # ENTER END VALUE FOR P1 RANGE HERE
        self.p1_max = self.max_num_1 * float(eval(self.model.get_parameter(self.p1).expression))
        # ENTER THE NUMBER OF STEPS FOR P1 HERE
        self.p1_steps = p1_steps
        self.p1_range = np.linspace(self.p1_min, self.p1_max, self.p1_steps)
        # ENTER START VALUE FOR P2 RANGE HERE
        self.p2_min = self.min_num_2 * float(eval(self.model.get_parameter(self.p2).expression))
        # ENTER END VALUE FOR P2 RANGE HERE
        self.p2_max = self.max_num_2 * float(eval(self.model.get_parameter(self.p2).expression))
        # ENTER THE NUMBER OF STEPS FOR P2 HERE
        self.p2_steps = p2_steps
        self.p2_range = np.linspace(self.p2_min, self.p2_max, self.p2_steps)

        # ENTER START VALUE FOR P3 RANGE HERE
        self.p3_min = self.min_num_3 * float(eval(self.model.get_parameter(self.p3).expression))
        # ENTER END VALUE FOR P3 RANGE HERE
        self.p3_max = self.max_num_3 * float(eval(self.model.get_parameter(self.p3).expression))
        # ENTER THE NUMBER OF STEPS FOR P3 HERE
        self.p3_steps = p3_steps
        self.p3_range = np.linspace(self.p3_min, self.p3_max, self.p3_steps)

        # ENTER VARIABLE OF INTEREST HERE
        self.variable_of_interest = ['fox','rabbit']
        # What feature of the simulation are we examining
        self.feature_extraction = population_at_last_timepoint
        # for ensemble resutls: how do we aggreggate the values
        self.ensemble_aggragator = average_of_ensemble

timespan: 10,20,30,40,50 

n_tra: 1,3,5,7
r_birth: [1,100]
f_birth: [0.001,0.1]
f_death: [1,100]

In [12]:
with open("fox_rabbit3D_multipara.csv", 'w', encoding='UTF8', newline = "") as f:
        writer = csv.writer(f)
        row = ["runtime", "The number of rabbits", "The number of foxes", "r_birth", "f_birth", "f_death", "number_of_trajectories", "Timespan"]
        writer.writerow(row)

In [11]:
psnew_dask1 = []
for timesp in range(10,50,10):
    for n_tra in range(1,8,2):
        kwargs3D_new = configure_simulation(n_tra, timesp)
        for steps in range(10,21,10):
            for n in range(1, 6, 2):
                psnew = ParameterSweepConfig3D(n*0.5,n*2,n*0.5,n*2,n*0.5,n*2, steps, steps, steps, timesp, kwargs3D_new)
                psnew_dask1.append(psnew)

In [13]:
futures55 = client.map(run, psnew_dask1)

In [16]:
progress(futures55)

VBox()

2023-03-19 14:37:35,840 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:38:11,184 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:38:45,989 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:39:19,678 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:40:05,432 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:40:30,302 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:40:49,166 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:41:12,289 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:41:51,972 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:42:16,277 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:42:28,504 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:42:36,299 - GillesPy2 - WARNI

2023-03-19 14:54:08,696 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:54:09,702 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:54:14,134 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:54:16,643 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:54:29,922 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:54:34,157 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:54:36,786 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:54:48,538 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:54:49,842 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:54:52,017 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:55:02,413 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 14:55:07,155 - GillesPy2 - WARNI

2023-03-19 15:01:51,658 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:01:53,045 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:01:59,102 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:02:00,487 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:02:04,384 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:02:06,558 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:02:11,678 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:02:21,815 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:02:23,629 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:02:27,227 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:02:30,081 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:02:31,713 - GillesPy2 - WARNI

2023-03-19 15:08:37,481 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:08:43,827 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:08:50,353 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:08:52,767 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:08:56,530 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:09:01,514 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:09:03,162 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:09:05,278 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:09:07,204 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:09:10,385 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:09:12,790 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 15:09:16,056 - GillesPy2 - WARNI

...

2023-03-19 18:43:32,417 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:43:38,620 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:43:43,466 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:43:55,326 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:43:58,634 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:44:07,476 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:44:18,077 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:44:18,655 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:44:29,710 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:44:38,095 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:44:42,494 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:44:53,992 - GillesPy2 - WARNI

2023-03-19 18:58:45,326 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:58:57,281 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:59:05,345 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:59:17,296 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:59:29,263 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:59:37,327 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:59:49,281 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 18:59:57,344 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:00:13,182 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:00:17,366 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:00:33,201 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:00:37,383 - GillesPy2 - WARNI

2023-03-19 19:20:38,754 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:20:58,785 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:21:18,814 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:21:38,849 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:22:02,695 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:22:22,726 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:22:42,758 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:23:02,788 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:23:22,824 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:23:42,853 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:24:02,871 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:24:22,901 - GillesPy2 - WARNI

2023-03-19 19:53:59,140 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:54:19,165 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:54:39,189 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:54:59,215 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:55:19,238 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:55:39,267 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:55:59,311 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:56:19,354 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:56:43,264 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:57:03,298 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:57:23,330 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 19:57:43,355 - GillesPy2 - WARNI

2023-03-19 20:28:22,127 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:28:42,154 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:29:02,176 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:29:22,199 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:29:42,225 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:30:02,248 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:30:22,276 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:30:42,316 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:31:02,336 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:31:22,361 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:31:42,385 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2023-03-19 20:32:02,417 - GillesPy2 - WARNI

In [17]:
for future in futures55:
    result = future.result()
    result.write_to_file("fox_rabbit3D_multipara.csv")

In [27]:
client.shutdown()